In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import pickle, scipy, random, time
import matplotlib.pyplot as plt
%matplotlib inline
# plt.style.use('fivethirtyeight')

#import all necessary libraries

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.svm import SVC
import catboost, lightgbm, xgboost
from sklearn.naive_bayes import GaussianNB
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

## feature selection

In [36]:
df = pickle.load(open('feature1.pickle', 'rb'))
df['label'] = df['label'].astype('int')
df

,User_id,start,finish,label,wave_appro_x_1,wave_details_x_1,wave_appro_x_2,wave_details_x_2,wave_appro_x_3,wave_details_x_3,...,jerk_y_z_covar,gra_x_y_covar,gra_x_z_covar,gra_y_z_covar,body_x_y_covar,body_x_z_covar,body_y_z_covar,jerk_body_x_y_covar,jerk_body_x_z_covar,jerk_body_y_z_covar
0,12,2018-05-02 00:57:13.026000+00:00,2018-05-02 00:57:17.976000+00:00,1,29.575164,-0.074384,29.671432,0.113113,30.699674,0.124029,...,-0.004382,-0.000645,0.000002,-3.839830e-06,1.996606e-05,3.428637e-08,-2.192912e-07,0.000406,0.000094,-0.000040
1,12,2018-05-02 00:57:15.026000+00:00,2018-05-02 00:57:19.976000+00:00,1,31.223481,0.045835,31.093535,-0.333277,30.619536,0.157530,...,-0.008945,-0.000416,0.000005,5.423978e-08,-6.619763e-04,-2.515746e-05,-3.416138e-05,0.055622,-0.012920,-0.016635
2,12,2018-05-02 00:57:17.026000+00:00,2018-05-02 00:57:21.976000+00:00,1,30.414322,-0.001846,30.334898,-0.302524,30.927157,0.352237,...,-0.005040,-0.000241,0.000008,2.851753e-06,2.054279e-04,-2.511390e-05,-1.920471e-05,0.114467,-0.012324,-0.011097
3,12,2018-05-02 00:57:19.026000+00:00,2018-05-02 00:57:23.976000+00:00,1,29.984322,-0.013482,30.010792,0.054172,30.644166,0.088239,...,-0.003277,-0.000116,0.000008,4.399206e-06,-1.843823e-04,6.526186e-06,9.306546e-06,0.038879,-0.004096,0.000005
4,12,2018-05-02 00:57:21.026000+00:00,2018-05-02 00:57:25.976000+00:00,1,30.414296,-0.012291,30.317877,-0.400519,31.197177,0.485815,...,-0.003977,-0.000035,0.000007,4.824796e-06,-3.911691e-04,-1.410523e-05,1.495776e-05,-0.015768,-0.007513,-0.004358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130345,4,2018-06-13 23:41:23.002000+00:00,2018-06-13 23:41:27.952000+00:00,28,-103.089534,-0.015961,-103.087421,-0.044936,-102.760547,0.101208,...,-0.000122,-0.000017,0.000002,1.030971e-06,3.080432e-04,1.179615e-05,6.513816e-06,0.008815,0.000514,0.001227
130346,4,2018-06-13 23:41:25.002000+00:00,2018-06-13 23:41:29.952000+00:00,28,-102.676474,0.012444,-102.716918,-0.109757,-102.868489,0.052805,...,-0.000036,-0.000029,0.000007,2.633280e-06,1.399728e-04,5.067773e-05,5.074720e-06,0.024472,-0.000123,0.001254
130347,4,2018-06-13 23:41:27.002000+00:00,2018-06-13 23:41:31.952000+00:00,28,-103.240176,-0.009438,-103.191811,0.149265,-103.088257,-0.092804,...,-0.001985,-0.000047,0.000017,8.933175e-06,5.636844e-04,-7.243138e-06,5.646384e-05,0.016200,-0.001231,-0.001264
130348,4,2018-06-13 23:41:29.002000+00:00,2018-06-13 23:41:33.952000+00:00,28,-103.156734,-0.011518,-103.157193,-0.039833,-102.799585,0.103600,...,-0.002847,-0.000071,0.000033,2.131977e-05,1.480436e-04,2.760928e-05,-3.917918e-05,-0.002155,0.000596,-0.004434


In [6]:
def feature_selection(n_features):
    
    etc = ExtraTreesClassifier(n_estimators=50, max_depth=13, n_jobs=-1, criterion='entropy').fit(df.drop(columns=['User_id', 'start', 'finish', 'label']), df['label'])
    rf = RandomForestClassifier(n_estimators=60, max_depth=13, n_jobs=-1, criterion='entropy').fit(df.drop(columns=['User_id', 'start', 'finish', 'label']), df['label'])

    cols = df.drop(columns=['User_id', 'start', 'finish', 'label']).columns
    etc_fi = pd.Series(etc.feature_importances_, index=cols)
    rf_fi = pd.Series(rf.feature_importances_, index=cols)

    treefi = (etc_fi.sort_index() + rf_fi.sort_index())/2
    treefi = treefi.sort_values()
    return treefi.tail(n_features).index

remained_columns = feature_selection(280)


In [37]:
remained_columns = pickle.load(open('remained_columns320.pkl', 'rb'))
len(remained_columns)

320

## read data

In [38]:
df = df[['User_id', 'label'] + list(remained_columns)]
df.dropna(inplace=True)
# df.drop(columns=['User_id'], inplace=True)
df['label'] = df['label'].astype('int')
# df.drop(columns=list(del_cols), inplace=True)
df.shape

C:\Users\User\.conda\envs\AntonioEnv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\User\.conda\envs\AntonioEnv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


(130349, 322)

## under and over sampling

In [40]:
def under_sampling(df):
    labels = [4,2,14]
    n_rows = [33000, 24000, 8000]

    for label, rows in zip(labels, n_rows):
        ind = df[df['label']==label].index
        r = random.sample(range(0, len(ind)), rows)
        deleted = ind[r]
        df.drop(deleted, axis=0, inplace=True)
    print(df.shape)
    return df

In [41]:
def over_sampling(df):
    # !pip install imblearn
    from imblearn.over_sampling import RandomOverSampler
    os = RandomOverSampler(random_state=25)
    labels1 = df['label'].value_counts().head(11).index
    
    df1 = df[df['label'].isin(labels1)]
    labels2 = df['label'].value_counts().tail(13).index
    df2 = df[df['label'].isin(labels2)]
    
    X1, y1 = df1.drop(columns=['label']), df1['label']
    X2, y2 = os.fit_resample(df2.drop(columns=['label']), df2['label'])

    print(f'after undersampling(topclass data) : {X1.shape, y1.shape}')
    print(f'after oversampling(underclass data) : {X2.shape, y2.shape}')

    X = pd.concat([X1, X2], axis=0)
    y = pd.concat([y1, y2], axis=0)

    print(f'shape of my training data : {X.shape, y.shape}')
    return X,y

In [42]:
dff = under_sampling(df.drop(columns=['User_id']))
X, y = over_sampling(dff)

dff = pd.concat([X,y], axis=1)

dff = dff.sample(frac=1).reset_index(drop=True)
X_train = dff.drop(columns=['label'])
y_train = dff['label']
print(X_train.shape, y_train.shape)
print()

(65349, 321)
after undersampling(topclass data) : ((63324, 320), (63324,))
after oversampling(underclass data) : ((10998, 320), (10998,))
shape of my training data : ((74322, 320), (74322,))
(74322, 320) (74322,)



In [43]:
y_train.value_counts()

4     10677
2     10645
14    10119
12     9766
9      6408
10     5025
7      2812
16     2447
13     2440
19     1719
11     1266
3       846
5       846
6       846
8       846
28      846
25      846
15      846
17      846
18      846
20      846
22      846
24      846
1       846
Name: label, dtype: int64

## model_train

In [30]:
def train_model(model):
    ini = time.time()
    clf = model.fit(X_train,y_train)
    cv_score = cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy')

#     print(f'model info for {model_name.upper()} classifier : ')
    print('---------------------------------------')
    print(f'Cross-validation : {cv_score}')
    print(f'mean : {np.mean(cv_score)}')
#     print(f'median : {np.median(cv_score)}')
#     print(f'std : {np.std(cv_score)}\n')
    print(f'shape of input data : {X_train.shape}')

In [378]:
train_model(RandomForestClassifier(n_estimators=600, max_depth=13, n_jobs=-1, criterion='entropy'))

---------------------------------------
Cross-validation : [0.87350835 0.87719059 0.87397709 0.87602291 0.87370431]
mean : 0.874880650587247
shape of input data : (73322, 280)


In [363]:
train_model(lightgbm.LGBMClassifier(n_estimators=200, reg_lambda=0.01, reg_alpha=0.001))

---------------------------------------
Cross-validation : [0.87241732 0.87241732 0.87397709 0.87697763 0.87343153]
mean : 0.8738441784691722
shape of input data : (73322, 280)


## split_user

In [52]:
train_df = df[df['User_id'].isin([2,3,4,5,6,12,17])]
X_train, y_train = train_df.drop(columns=['User_id', 'label']), train_df['label']
print(train_df.shape)
print(X_train.shape, y_train.shape)

(98910, 322)
(98910, 320) (98910,)


In [53]:
val_df = df[~df['User_id'].isin([2,3,4,5,6,12,17])]
X_val, y_val = val_df.drop(columns=['User_id', 'label']), val_df['label']
print(val_df.shape)
print(X_val.shape, y_val.shape)

(31439, 322)
(31439, 320) (31439,)


In [54]:
y_train.value_counts()

4     40535
2     17851
14    11087
12     9766
9      5141
10     4538
7      2812
16     2447
13     2412
1       772
19      363
5       182
17      154
6       148
22      134
3       126
28      106
18       89
8        82
24       57
15       55
11       35
25       17
20        1
Name: label, dtype: int64

In [55]:
train_df = train_df.drop(columns=['User_id'])
val_df = val_df.drop(columns=['User_id'])

## under and over sampling for user_split

In [56]:
def under_sampling1(df):
    labels = [4,2]
    n_rows = [28000, 5000]

    for label, rows in zip(labels, n_rows):
        ind = df[df['label']==label].index
        r = random.sample(range(0, len(ind)), rows)
        deleted = ind[r]
        df.drop(deleted, axis=0, inplace=True)
    print(df.shape)
    return df

def over_sampling1(df):
    # !pip install imblearn
    from imblearn.over_sampling import RandomOverSampler
    os = RandomOverSampler(random_state=42)
    labels1 = df['label'].value_counts().head(9).index
    
    df1 = df[df['label'].isin(labels1)]
    labels2 = df['label'].value_counts().tail(15).index
    df2 = df[df['label'].isin(labels2)]
    
    X1, y1 = df1.drop(columns=['label']), df1['label']
    X2, y2 = os.fit_resample(df2.drop(columns=['label']), df2['label'])

    print(f'after undersampling(topclass data) : {X1.shape, y1.shape}')
    print(f'after oversampling(underclass data) : {X2.shape, y2.shape}')

    X = pd.concat([X1, X2], axis=0)
    y = pd.concat([y1, y2], axis=0)

    print(f'shape of my training data : {X.shape, y.shape}')
    return X,y

In [57]:
dff = under_sampling1(train_df)
X, y = over_sampling1(train_df)

dff = pd.concat([X,y], axis=1)
dff = dff.sample(frac=1).reset_index(drop=True)
X = dff.drop(columns=['label'])
y = dff['label']
print(X.shape, y.shape)

(65910, 321)
after undersampling(topclass data) : ((63589, 320), (63589,))
after oversampling(underclass data) : ((11580, 320), (11580,))
shape of my training data : ((75169, 320), (75169,))
(75169, 320) (75169,)


In [58]:
y.value_counts()

2     12851
4     12535
14    11087
12     9766
9      5141
10     4538
7      2812
16     2447
13     2412
3       772
5       772
6       772
8       772
28      772
11      772
25      772
15      772
17      772
18      772
19      772
20      772
22      772
24      772
1       772
Name: label, dtype: int64

## model selection and hyperparameters tuning (manually)

In [59]:
# def lgbm_model(lmbda):
# lambda=0.01, alpha=0.001
ini = time.time()
clf = lightgbm.LGBMClassifier(n_estimators=200, reg_lambda=0.01, reg_alpha=0.001).fit(X, y)
final = time.time()
print(f'time required : {(final-ini)/60} minutes')
print(f'train score : {clf.score(X, y)}')
print(f'validation score : {clf.score(X_val, y_val)}')

time required : 8.246294085184733 minutes
train score : 0.9832111641767218
validation score : 0.4858615095899997


In [388]:
# def lgbm_model(lmbda):
ini = time.time()
clf = RandomForestClassifier(n_estimators=600, max_depth=13, n_jobs=-1, criterion='entropy').fit(X, y)
# RandomForestClassifier(n_estimators=600, max_depth=13, n_jobs=-1, criterion='entropy')
print(f'alpha={lmbda} : ')
final = time.time()
print(f'time required : {(final-ini)/60} minutes')
print(f'train score : {clf.score(X, y)}')
print(f'validation score : {clf.score(X_val, y_val)}')
print()

alpha=0.1 : 
time required : 24.767437314987184 minutes
train score : 0.9305097874463364
validation score : 0.3422818791946309



## more methods of feature selection

In [ ]:
# mutual info classif

from sklearn.feature_selection import mutual_info_classif
m_info = mutual_info_classif(X,y)

data = pd.Series(m_info, index=df.columns)
data

In [ ]:
# anova test

from sklearn.feature_selection import f_classif
anova = f_classif(X, y)
anova, _ = anova

a = pd.Series(anova, index=X.columns).sort_values()

In [ ]:
# chi_square test

chi2, _ = scipy.stats.chisquare(df.drop(columns=['label']))
b = pd.Series(chi2, index=df.drop(columns=['label']).columns).sort_values()

In [ ]:
# using high variance (the columns have lower variance, are not so significant for our model's performance)

var = []
for col in X.columns:
    var.append((np.std(X[col]))**2)
var_thres = pd.Series(var, index=X.columns).sort_values()